# Django 初始化

In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-03-03 12:27:18,304 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
#导入所有模型
from django.contrib.auth import get_user_model
from apps.files.models import FileRecord
from apps.projects.models import Project, ProjectHistory
from apps.doc_analysis.models import DocumentAnalysis

User = get_user_model()
projects = Project.objects.all()  #事实上可以直接使用
files = FileRecord.objects.all()
analyses = DocumentAnalysis.objects.all()


数据库数据清理，只保留18501771516用户的数据


In [3]:
# 删除18501771516用户以外的项目
deleted_count = Project.objects.exclude(creator__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他项目")


# 删除18501771516用户以外的文件
deleted_count = FileRecord.objects.exclude(owner__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他文件")

# 删除18501771516用户以外的分析
deleted_count = DocumentAnalysis.objects.exclude(created_by__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他分析")

# 删除18501771516用户以外的用户
deleted_count = User.objects.exclude(phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他用户")

已删除 0 个其他项目
已删除 0 个其他文件
已删除 0 个其他分析
已删除 0 个其他用户


# <font color="red">Part 1 - 用户管理</font>


In [4]:
# 用户模型所有字段
print("User")
print("-----------常规字段--------------")
for field in User._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in User._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in User._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")



User
-----------常规字段--------------
- password: CharField
- last_login: DateTimeField
- is_superuser: BooleanField
- id: UUIDField
- phone: CharField
- email: EmailField
- username: CharField
- wechat_id: CharField
- role: CharField
- is_active: BooleanField
- is_staff: BooleanField
- created_at: DateTimeField
- updated_at: DateTimeField

-----------反向关系字段-------------
- logentry_set: ManyToOneRel (来自 LogEntry)
- owned_files: ManyToOneRel (来自 FileRecord)
- created_projects: ManyToOneRel (来自 Project)
- created_analyses: ManyToOneRel (来自 DocumentAnalysis)
- confirmed_analyses: ManyToOneRel (来自 DocumentAnalysis)
- chat_sessions: ManyToOneRel (来自 ChatSession)
- outstandingtoken_set: ManyToOneRel (来自 OutstandingToken)

-----------多对多字段-------------
- groups: ManyToManyField
- user_permissions: ManyToManyField


### 打印所有用户

In [5]:
# 打印用户列表
users = User.objects.all()
for user in users:
    print(f"用户：{user.phone}," 
          f"项目数：{user.created_projects.all().count()}," 
          f"文件数：{user.owned_files.all().count()}",
          f"分析数：{user.created_analyses.all().count()}"
          )


用户：18501771516,项目数：3,文件数：0 分析数：0


### 过滤用户

In [6]:
# 打印用户的所有字段值
for user in User.objects.filter(phone='18501771516'):
    print(f"用户ID: {user.id}, 用户名: {user.username}")
    # 打印用户的所有字段值

    for field in User._meta.fields:
        print(f"  {field.name}: {getattr(user, field.name)}")
    print("---")



用户ID: bc58212c-453f-415f-8614-bef6d3f11923, 用户名: None
  password: 
  last_login: None
  is_superuser: False
  id: bc58212c-453f-415f-8614-bef6d3f11923
  phone: 18501771516
  email: None
  username: None
  wechat_id: None
  role: user
  is_active: True
  is_staff: False
  created_at: 2025-02-28 16:26:16.203678+00:00
  updated_at: 2025-02-28 16:26:16.203825+00:00
---


In [7]:
# 删除selected之外的所有用户
selected_phones = ['18501771516', '13800138000']
deleted_count = User.objects.exclude(phone__in=selected_phones).delete()
print(f"已删除 {deleted_count[0]} 个其他用户")


已删除 0 个其他用户


In [8]:
# 清空所有上传的文件
#delete_count = User.objects.all().delete()
#print(delete_count[0])

# <font color="red">Part 2 - 项目管理</font>


In [6]:
# 项目模型所有字段
print("Project")
print("-----------常规字段--------------")
for field in Project._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in Project._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in Project._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

Project
-----------常规字段--------------
- id: UUIDField
- project_code: CharField
- project_name: CharField
- tenderee: CharField
- bidder: CharField
- project_type: CharField
- bid_deadline: DateTimeField
- status: CharField
- current_stage: CharField
- is_urgent: BooleanField
- creator: ForeignKey
- create_time: DateTimeField
- last_update_time: DateTimeField

-----------反向关系字段-------------
- files: ManyToOneRel (来自 FileRecord)
- stage_histories: ManyToOneRel (来自 ProjectHistory)
- document_analyses: ManyToOneRel (来自 DocumentAnalysis)

-----------多对多字段-------------


In [8]:
# 打印项目列表
projects = Project.objects.all()
for project in projects:
    print(f"项目编号:{project.project_code}",
          f"项目名：{project.project_name}," 
          f"创建者：{project.creator.phone}," 
          f"关联分析数：{project.document_analyses.all().count()}"
          )

项目编号:BP-2025-OTHER-0001 项目名：阿里巴巴年节福利2,创建者：18501771516,关联分析数：0
项目编号:BP-2025-FSD-0001 项目名：xiangmu 1,创建者：18501771516,关联分析数：0
项目编号:BP-2025-WELFARE-0001 项目名：测试项目1,创建者：18501771516,关联分析数：0


In [11]:
# 清空所有上传的文件
#delete_count = Project.objects.all().delete()
#print(delete_count[0])

In [9]:
# 打印特定用户的所有项目的所有字段值
for project in Project.objects.filter(creator__phone='18501771516',project_code='BP-2025-WELFARE-0001'):
    print(f"项目ID: {project.id}, 项目名: {project.project_name}")
    # 打印项目的所有字段值

    for field in project._meta.fields:
        print(f"  {field.name}: {getattr(project, field.name)}")
    print("---")



项目ID: aae5a639-a4f7-4f01-91f1-b810672b1d9e, 项目名: 测试项目1
  id: aae5a639-a4f7-4f01-91f1-b810672b1d9e
  project_code: BP-2025-WELFARE-0001
  project_name: 测试项目1
  tenderee: 执智者
  bidder: 
  project_type: WELFARE
  bid_deadline: None
  status: IN_PROGRESS
  current_stage: INITIALIZATION
  is_urgent: False
  creator: 18501771516
  create_time: 2025-02-28 16:34:13.928284+00:00
  last_update_time: 2025-02-28 16:34:13.928284+00:00
---


# <font color="red">Part 3 - 文件管理</font>


In [13]:
# 文件所有字段
# 文件的所有常规字段
print("FileRecord")
print("-----------常规字段--------------")
for field in FileRecord._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in FileRecord._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in FileRecord._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

FileRecord
-----------常规字段--------------
- id: UUIDField
- created_at: DateTimeField
- created_by: CharField
- updated_at: DateTimeField
- updated_by: CharField
- version: IntegerField
- name: CharField
- file: FileField
- size: BigIntegerField
- type: CharField
- mime_type: CharField
- processing_status: CharField
- processing_progress: IntegerField
- error_message: TextField
- owner: ForeignKey
- metadata: JSONField
- remarks: TextField
- project: ForeignKey

-----------反向关系字段-------------
- document_analyses: ManyToOneRel (来自 DocumentAnalysis)

-----------多对多字段-------------


In [15]:
# 打印项目列表
files = FileRecord.objects.all()
for file in files:
    print(f"文件ID:{file.id}",
          f"文件类型:{file.type}",
          f"文件名：{file.name}," 
          f"拥有者：{file.owner.phone}," 
          f"关联分析数：{file.document_analyses.all().count()}"
          )

文件ID:cf182d9b-b3ff-4422-92f1-47eb3ff0f5ce 文件类型:WORD 文件名：实战：招标文件-第1包：一级压榨花生油.docx,拥有者：18501771516,关联分析数：1


In [18]:
# 清空所有上传的文件
#delete_count = FileRecord.objects.all().delete()
#print(delete_count[0])

In [19]:
# 打印特定用户的所有项目的所有字段值
for file in FileRecord.objects.filter(owner__phone='18501771516',name='Test 文本.docx'):
    print(f"文件ID: {file.id}, 文件名: {file.name}")
    # 打印文件的所有字段值


    for field in file._meta.fields:
        print(f"  {field.name}: {getattr(file, field.name)}")
    print("---")

<font color="red">将文件关联到项目</font>


In [17]:
#delete_count = FileProjectLink.objects.filter(project=project).delete()
#print(deleted_count[0])

# <font color=red>Part 4 - 分析管理 </font>

In [13]:
# 文档分析的所有字段
# 常规字段
print("DocumentAnalysis")
print("-----------常规字段--------------")
for field in DocumentAnalysis._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in DocumentAnalysis._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in DocumentAnalysis._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

DocumentAnalysis
-----------常规字段--------------
- id: UUIDField
- project: ForeignKey
- file_record: ForeignKey
- title: CharField
- status: CharField
- analysis_questions: JSONField
- analysis_result: JSONField
- extracted_elements: JSONField
- outline_analysis_result: JSONField
- improved_docx_elements: JSONField
- docxtree: JSONField
- more_subtitles: JSONField
- error_message: TextField
- raw_xml: TextField
- processing_time: DurationField
- analysis_version: CharField
- created_at: DateTimeField
- updated_at: DateTimeField
- analyzed_at: DateTimeField
- confirmed_at: DateTimeField
- created_by: ForeignKey
- confirmed_by: ForeignKey

-----------反向关系字段-------------

-----------多对多字段-------------


In [14]:
analyses = DocumentAnalysis.objects.all()

In [15]:
print(analyses)

<QuerySet []>


In [16]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n")

In [22]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n",
          f"分析名称：{analysis.title}\n",
          f"分析所在项目：{analysis.project.project_name}\n", 
          f"分析的文件：{analysis.file_record.name}\n" ,
         # f"分析的阶段：{analysis.status}\n",
         # f"分析的问题：{analysis.analysis_questions}\n",
         # f"分析结果：{analysis.analysis_result}\n",
         # f"分析创建者：{analysis.created_by.phone}\n",
         # f"分析用时：{analysis.processing_time}\n",
          )

In [23]:
delete_count = DocumentAnalysis.objects.all().delete()
print(delete_count[0])

0
